# Graph Reports API Module - OEA Connector
This notebook demonstrates the utility of the OEA_py class notebook, by supporting the Graph module pipeline. 

There are currently 4 accepted, overarching methods for the ```method``` parameter (see bottom code block for seeing how this notebook handles the methods):
 1. **land_testdata**: Lands either the K-12 or Higher Ed. Graph test datasets.
 2. **preprocess_meeting**: Pre-processes and cleans the meeting_attendance_report table, if the higher ed test dataset has been chosen for processing.
 3. **correct_schemas**: Corrects the Graph schema based on the test dataset ingested.
 4. **refine_graph**: Refines the Graph tables based on the test dataset ingested.

There are currently 2 accepted inputs for the ```testdataSet``` parameter: 
 1. **k12**: processes Graph module K-12 test data.
 2. **hed**: processes Graph module higher education test data.

Refer to each function markdown header for comments and general function outline.

In [ ]:
# parameters fed to the notebook from the pipeline
workspace = 'dev'
testdataSet = 'hed'
method = ''

In [ ]:
%run /OEA_py

In [ ]:
# 1) set the workspace (this determines where in the data lake you'll be writing to and reading from).
# You can work in 'dev', 'prod', or a sandbox with any name you choose.
# For example, Sam the developer can create a 'sam' workspace and expect to find his datasets in the data lake under oea/sandboxes/sam
oea.set_workspace(workspace)

## 1.) Land Graph Testdata

This function is used to speed up the landing of test data, since there are ~60 JSON files for the K-12 test dataset, and ~320 JSON files for the higher ed. test dataset.

```method```: **land_testdata**

 - If ```testdataSet``` is ```k12```, land the K-12 test dataset using the oea.land() function.
    * **Lands 3 beta-query tables**: users, m365_app_user_detail, teams_activity_user_detail.
 - If ```testdataSet``` is ```hed```, land the higher ed. test dataset using the oea.land() function.
    * **Lands 3 beta-query tables**: users, m365_app_user_detail, teams_activity_user_detail.
    * **Lands 1 v1.0-query table**: meeting_attendance_report.

In [ ]:
# a.) first function used - lands the desired Graph module test dataset.
def _k12_testdata():
    import datetime
    currentDate = datetime.datetime.now()
    currentDateTime = currentDate.strftime("%Y-%m-%d %H-%M-%S")
    data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/k12_test_data/Users/part-00000-cae42818-3572-4824-b396-58587ad01616-c000.json').text
    oea.land(data, 'graph_api/beta/users', 'users_k12_test_data.json', oea.SNAPSHOT_BATCH_DATA, currentDateTime)
    # iteratively land m365 and teams files
    m = 0
    for n in 31:
        if n == 0:
            nstring='00'
            m+=1
        elif n==1:
            nstring='01'
            m+=1
        elif n==2:
            nstring='02'
            m+=1
        elif n==3:
            nstring='03'
            m+=1
        elif n==4:
            nstring='04'
            m+=1
        elif n==5:
            nstring='05'
            m+=1
        elif n==6:
            nstring='06'
            m+=1
        elif n==7:
            nstring='07'
            m+=1
        elif n==8:
            nstring='08'
            m+=1
        elif n==9:
            nstring='09'
            m+=1
        else:
            nstring = str(m)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/k12_test_data/M365_App_User_Detail/part-000'+nstring+'-b6c697c6-1520-4aae-9f5c-6213dff6ab0a-c000.json').text
        oea.land(data, 'graph_api/beta/m365_app_user_detail', 'part'+nstring+'_k12_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/k12_test_data/Teams_Activity_User_Detail/part-000'+nstring+'-bd366e94-d20e-4e81-b4ba-47b15889b4a9-c000.json').text
        oea.land(data, 'graph_api/beta/teams_activity_user_detail', 'part'+nstring+'_k12_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)

def _hed_testdata():
    import datetime
    currentDate = datetime.datetime.now()
    currentDateTime = currentDate.strftime("%Y-%m-%d %H-%M-%S")
    data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Users/part-00000-cae42818-3572-4824-b396-58587ad01616-c000.json').text
    oea.land(data, 'graph_api/beta/users', 'users_hed_test_data.json', oea.SNAPSHOT_BATCH_DATA, currentDateTime)
    # iteratively land m365 and teams files
    m=0
    for n in 31:
        if n < 10:
            nstring = str(m)
            nstring += '0'
        else:
            nstring = str(m)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/M365_App_User_Detail/part-000'+nstring+'-b6c697c6-1520-4aae-9f5c-6213dff6ab0a-c000.json').text
        oea.land(data, 'graph_api/beta/m365_app_user_detail', 'part'+nstring+'_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Teams_Activity_User_Detail/part-000'+nstring+'-bd366e94-d20e-4e81-b4ba-47b15889b4a9-c000.json').text
        oea.land(data, 'graph_api/beta/teams_activity_user_detail', 'part'+nstring+'_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)

def land_graph_testdata(testdataset='hed'):
    import datetime
    currentDate = datetime.datetime.now()
    currentDateTime = currentDate.strftime("%Y-%m-%d %H-%M-%S")
    if testdataset == 'k12':
        # a.1) Land batch data files into stage1 of the data lake.
        # pull Graph K-12 test json data files from github and land it in oea/sandboxes/sam/stage1/Transactional/graph_api/beta
        _k12_testdata()
        
            
    elif testdataset == 'hed':
        # a.2) Land batch data files into stage1 of the data lake.
        # In this example we pull Graph Higher Ed. test json data files from github and land it in oea/sandboxes/sam/stage1/Transactional/graph_api/beta 
        _hed_testdata()
        
        # a.25) Land batch data files into stage1 of the data lake.
        # In this example we pull Graph Higher Ed. test json data files from github and land it in oea/sandboxes/sam/stage1/Transactional/graph_api/v1.0
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/030733285E7943CE86189E39C88A04D6_13_16.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report001_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/030733285E7943CE86189E39C88A04D6_20_15.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report002_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/030733285E7943CE86189E39C88A04D6_21_12.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report003_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/03A2BF1EEE524B1CADC97E214963BB92_11_14.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report004_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/03A2BF1EEE524B1CADC97E214963BB92_14_9.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report005_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/03A2BF1EEE524B1CADC97E214963BB92_6_15.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report006_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/08DDFD65A495417587EE3E91F25FEECD_20_15.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report007_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/08DDFD65A495417587EE3E91F25FEECD_21_15.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report008_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/08DDFD65A495417587EE3E91F25FEECD_28_12.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report009_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/16D3783DEA504F35B66FD00178DFC2CB_13_11.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report010_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/16D3783DEA504F35B66FD00178DFC2CB_17_10.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report011_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/16D3783DEA504F35B66FD00178DFC2CB_25_15.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report012_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/1AC9DCA1874A42B0915A699B7675B90F_10_10.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report013_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/1AC9DCA1874A42B0915A699B7675B90F_17_11.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report014_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/1AC9DCA1874A42B0915A699B7675B90F_26_13.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report015_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/1CC8AEF1B8364E2C8B1C00CEFA109248_10_15.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report016_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/1CC8AEF1B8364E2C8B1C00CEFA109248_20_14.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report017_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/1CC8AEF1B8364E2C8B1C00CEFA109248_26_15.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report018_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/1D636A19738249C59B08BB8D41E240F9_10_7.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report019_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/1D636A19738249C59B08BB8D41E240F9_20_14.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report020_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/1D636A19738249C59B08BB8D41E240F9_26_7.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report021_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/21A2ECF0DA464984BAA8056F9257AC03_24_13.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report022_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/21A2ECF0DA464984BAA8056F9257AC03_24_9.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report023_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/21A2ECF0DA464984BAA8056F9257AC03_4_12.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report024_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/2234ACFD1FE44510B19607B1D31AD714_10_7.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report025_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/2234ACFD1FE44510B19607B1D31AD714_24_9.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report026_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/2234ACFD1FE44510B19607B1D31AD714_4_7.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report027_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/2481F408C8FF44C99E64CF522F323EEF_11_10.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report028_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/2481F408C8FF44C99E64CF522F323EEF_12_8.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report029_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/2481F408C8FF44C99E64CF522F323EEF_18_9.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report030_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/25E91551DD0F4EB69FA10ABF1CE740B7_11_10.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report031_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/25E91551DD0F4EB69FA10ABF1CE740B7_21_11.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report032_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/25E91551DD0F4EB69FA10ABF1CE740B7_25_11.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report033_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/2A16BC7D7F0D4B90822FEB162FFFA3BF_11_16.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report034_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/2A16BC7D7F0D4B90822FEB162FFFA3BF_13_13.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report035_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/2A16BC7D7F0D4B90822FEB162FFFA3BF_20_13.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report036_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/346617E5E8AD4C5C96A345B428660558_18_12.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report037_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/346617E5E8AD4C5C96A345B428660558_21_12.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report038_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/346617E5E8AD4C5C96A345B428660558_25_10.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report039_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/3B4BBA143AF041839B597DFA97CA93C5_11_15.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report040_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/3B4BBA143AF041839B597DFA97CA93C5_18_9.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report041_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/3B4BBA143AF041839B597DFA97CA93C5_3_13.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report042_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/3BA7BDD4577C4974A1D46F51E9EA2831_21_10.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report043_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/3BA7BDD4577C4974A1D46F51E9EA2831_25_13.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report044_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/3BA7BDD4577C4974A1D46F51E9EA2831_4_12.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report045_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/3C0DCC5113564086A3726ADA0D329E71_10_8.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report046_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/3C0DCC5113564086A3726ADA0D329E71_12_15.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report047_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/3C0DCC5113564086A3726ADA0D329E71_4_13.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report048_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/3C97C00264E84D34BC36231432D26D08_11_12.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report049_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/3C97C00264E84D34BC36231432D26D08_24_9.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report050_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/3C97C00264E84D34BC36231432D26D08_3_10.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report051_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/3CFE40C9DD8246F1A6E574AA0CB2DC93_11_9.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report052_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/3CFE40C9DD8246F1A6E574AA0CB2DC93_27_7.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report053_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/3CFE40C9DD8246F1A6E574AA0CB2DC93_5_13.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report054_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/3DC76BEF0A674B14A3416A751E6614AD_19_7.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report055_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/3DC76BEF0A674B14A3416A751E6614AD_21_15.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report056_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/3DC76BEF0A674B14A3416A751E6614AD_26_9.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report057_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/4334F5A95C144FFF898B7721C0DCCFF6_11_15.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report058_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/4334F5A95C144FFF898B7721C0DCCFF6_11_7.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report059_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/4334F5A95C144FFF898B7721C0DCCFF6_18_8.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report060_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/4394AC06CD484BCF97BA17DE0076B2A2_19_16.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report061_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/4394AC06CD484BCF97BA17DE0076B2A2_28_10.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report062_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/4394AC06CD484BCF97BA17DE0076B2A2_28_7.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report063_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/445E913608A44FBDA99B8108E1990EB0_13_7.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report064_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/445E913608A44FBDA99B8108E1990EB0_17_9.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report065_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/445E913608A44FBDA99B8108E1990EB0_18_7.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report066_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/4659EDDC864346B9BE42E52321A14D3F_11_13.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report067_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/4659EDDC864346B9BE42E52321A14D3F_14_12.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report068_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/4659EDDC864346B9BE42E52321A14D3F_7_10.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report069_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/49589E18A4734BB182FEFE0F8B8D6962_21_11.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report070_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/49589E18A4734BB182FEFE0F8B8D6962_3_14.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report071_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/49589E18A4734BB182FEFE0F8B8D6962_6_12.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report072_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/49A1942FADD6402DB745870761477CAF_10_9.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report073_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/49A1942FADD6402DB745870761477CAF_14_12.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report074_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/49A1942FADD6402DB745870761477CAF_19_10.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report075_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/508D26EBA720459FB44FA6C910832522_18_9.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report076_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/508D26EBA720459FB44FA6C910832522_28_13.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report077_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/508D26EBA720459FB44FA6C910832522_6_7.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report078_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/518053972F9E417990607B6F95279906_11_11.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report079_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/518053972F9E417990607B6F95279906_14_11.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report080_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/518053972F9E417990607B6F95279906_17_14.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report081_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/525498CDB2494D87A529C129A592BA75_20_14.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report082_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/525498CDB2494D87A529C129A592BA75_24_13.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report083_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/525498CDB2494D87A529C129A592BA75_3_13.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report084_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/53114219F05E4AAC877219B6D7181E7A_21_11.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report085_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/53114219F05E4AAC877219B6D7181E7A_26_16.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report086_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/53114219F05E4AAC877219B6D7181E7A_4_9.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report087_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/57A93AA78A6F492287C15F30F238BE1D_11_10.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report088_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/57A93AA78A6F492287C15F30F238BE1D_3_14.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report089_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/57A93AA78A6F492287C15F30F238BE1D_5_16.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report090_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/5D129FA96C454B0387237E799D1EEF12_28_10.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report091_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/5D129FA96C454B0387237E799D1EEF12_28_8.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report092_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/5D129FA96C454B0387237E799D1EEF12_5_8.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report093_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/5EECBA2B274847679615AEC649EFA200_27_15.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report094_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/5EECBA2B274847679615AEC649EFA200_28_13.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report095_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/5EECBA2B274847679615AEC649EFA200_5_11.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report096_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/66D505FBFEC5477684B7DFE66323C8CB_25_9.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report097_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/66D505FBFEC5477684B7DFE66323C8CB_4_8.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report098_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/66D505FBFEC5477684B7DFE66323C8CB_7_14.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report099_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/66EF3D922B054714AA9060ECFA1E50D0_12_12.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report100_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/66EF3D922B054714AA9060ECFA1E50D0_14_15.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report101_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/66EF3D922B054714AA9060ECFA1E50D0_7_11.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report102_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/69F85F73C3104AFE9E0427C808E3D2CD_20_8.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report103_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/69F85F73C3104AFE9E0427C808E3D2CD_25_7.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report104_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/69F85F73C3104AFE9E0427C808E3D2CD_5_16.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report105_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/6B2F0FEF9F38494FB10B3F1AB1FB0210_18_10.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report106_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/6B2F0FEF9F38494FB10B3F1AB1FB0210_4_13.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report107_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/6B2F0FEF9F38494FB10B3F1AB1FB0210_7_10.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report108_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/6F6FBD3F724F4CB8AE2F090B37BB729A_24_10.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report109_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/6F6FBD3F724F4CB8AE2F090B37BB729A_27_8.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report110_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/6F6FBD3F724F4CB8AE2F090B37BB729A_7_16.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report111_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/7307533D0C4143469F22436EF53FDD00_17_15.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report112_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/7307533D0C4143469F22436EF53FDD00_3_13.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report113_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/7307533D0C4143469F22436EF53FDD00_3_9.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report114_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/7510AF2BAFE14181BE9ED11FBBE569AB_17_15.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report115_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/7510AF2BAFE14181BE9ED11FBBE569AB_24_8.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report116_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/7510AF2BAFE14181BE9ED11FBBE569AB_28_12.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report117_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/78E67CD474B7442A8DA5297A09A558D8_17_14.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report118_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/78E67CD474B7442A8DA5297A09A558D8_27_12.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report119_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/78E67CD474B7442A8DA5297A09A558D8_5_9.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report120_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/7D851C14FFA244CF909340AC7863BAC3_28_14.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report121_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/7D851C14FFA244CF909340AC7863BAC3_5_11.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report122_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/7D851C14FFA244CF909340AC7863BAC3_7_9.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report123_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/7EA3285A7DFB4C2389A12D00CE77BDF9_10_8.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report124_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/7EA3285A7DFB4C2389A12D00CE77BDF9_11_7.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report125_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/7EA3285A7DFB4C2389A12D00CE77BDF9_5_10.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report126_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/7F9AE60E119947A587F733734913F561_11_8.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report127_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/7F9AE60E119947A587F733734913F561_12_14.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report128_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/7F9AE60E119947A587F733734913F561_7_9.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report129_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/856A511D6D124BE3A61D8A9FBECF89D1_11_13.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report130_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/856A511D6D124BE3A61D8A9FBECF89D1_11_15.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report131_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/856A511D6D124BE3A61D8A9FBECF89D1_18_7.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report132_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/88C5D9EE75344355ACBFAD211D436E0A_11_14.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report133_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/88C5D9EE75344355ACBFAD211D436E0A_11_16.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report134_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/88C5D9EE75344355ACBFAD211D436E0A_21_12.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report135_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/8F93CDB9FB7740899247AE8EE1B647AE_11_9.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report136_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/8F93CDB9FB7740899247AE8EE1B647AE_24_7.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report137_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/8F93CDB9FB7740899247AE8EE1B647AE_27_14.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report138_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/90A1DB4399014520B10AC7FBCC8A165C_14_8.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report139_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/90A1DB4399014520B10AC7FBCC8A165C_24_16.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report140_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/90A1DB4399014520B10AC7FBCC8A165C_5_16.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report141_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/92C670E8E11A47888A574BE6D6300042_10_12.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report142_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/92C670E8E11A47888A574BE6D6300042_13_12.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report143_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/92C670E8E11A47888A574BE6D6300042_14_9.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report144_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/95AE2468EFE544689808A2C3516E6B2E_13_9.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report145_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/95AE2468EFE544689808A2C3516E6B2E_27_11.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report146_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/95AE2468EFE544689808A2C3516E6B2E_3_15.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report147_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/95CB73976B544812B6CACDFAB76A4A7D_10_8.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report148_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/95CB73976B544812B6CACDFAB76A4A7D_4_13.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report149_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/95CB73976B544812B6CACDFAB76A4A7D_5_12.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report150_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/9EA9D757EBCD4C7BB1B65077F9F1AEF3_17_16.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report151_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/9EA9D757EBCD4C7BB1B65077F9F1AEF3_18_10.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report152_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/9EA9D757EBCD4C7BB1B65077F9F1AEF3_6_15.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report153_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/A12FDEF5617C49E380791A543353B0A2_20_8.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report154_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/A12FDEF5617C49E380791A543353B0A2_7_14.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report155_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/A12FDEF5617C49E380791A543353B0A2_7_16.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report156_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/A41A4D8CF7CD4CCCABA4FE5FC3CE21BF_11_12.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report157_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/A41A4D8CF7CD4CCCABA4FE5FC3CE21BF_28_7.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report158_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/A41A4D8CF7CD4CCCABA4FE5FC3CE21BF_7_12.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report159_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/A520F654FBCC4CCEBACEC63D2333D771_14_10.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report160_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/A520F654FBCC4CCEBACEC63D2333D771_17_12.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report161_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/A520F654FBCC4CCEBACEC63D2333D771_17_7.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report162_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/A7D4A05729F84E00B19FF5279BC0ACB2_13_15.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report163_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/A7D4A05729F84E00B19FF5279BC0ACB2_14_10.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report164_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/A7D4A05729F84E00B19FF5279BC0ACB2_5_8.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report165_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/AACE248D43BE4595AEEED5A95FA70ED2_28_13.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report166_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/AACE248D43BE4595AEEED5A95FA70ED2_4_15.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report167_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/AACE248D43BE4595AEEED5A95FA70ED2_5_13.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report168_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/AD1204DE0EE14ECB97A62A03310E1FA5_19_13.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report169_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/AD1204DE0EE14ECB97A62A03310E1FA5_26_12.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report170_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/AD1204DE0EE14ECB97A62A03310E1FA5_3_10.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report171_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/B207F983B9524E5F892884A8934D1077_21_11.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report172_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/B207F983B9524E5F892884A8934D1077_21_12.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report173_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/B207F983B9524E5F892884A8934D1077_24_7.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report174_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/B3890CDB0D9148C78770334E44F30729_19_10.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report175_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/B3890CDB0D9148C78770334E44F30729_19_15.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report176_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/B3890CDB0D9148C78770334E44F30729_21_16.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report177_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/B6E4ABF9F1CA41A6A5317AFB4BDCACB4_10_16.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report178_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/B6E4ABF9F1CA41A6A5317AFB4BDCACB4_13_10.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report179_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/B6E4ABF9F1CA41A6A5317AFB4BDCACB4_18_16.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report180_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/BE0A4AC2F0264375B4ED97DC17DD6800_13_10.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report181_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/BE0A4AC2F0264375B4ED97DC17DD6800_17_16.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report182_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/BE0A4AC2F0264375B4ED97DC17DD6800_25_8.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report183_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/BEB8D1707FEB453A85B387A9C474F08F_3_7.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report184_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/BEB8D1707FEB453A85B387A9C474F08F_5_7.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report185_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/BEB8D1707FEB453A85B387A9C474F08F_7_11.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report186_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/C6BFFB2D75F5425EAA743F666C89702B_10_16.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report187_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/C6BFFB2D75F5425EAA743F666C89702B_17_16.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report188_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/C6BFFB2D75F5425EAA743F666C89702B_26_11.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report189_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/C6C48F7DA7FB413AB903DF50BDA02C1D_24_13.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report190_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/C6C48F7DA7FB413AB903DF50BDA02C1D_5_12.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report191_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/C6C48F7DA7FB413AB903DF50BDA02C1D_6_7.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report192_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/C9314F1342D740C5B8C89FB12C55DBB8_10_12.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report193_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/C9314F1342D740C5B8C89FB12C55DBB8_18_16.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report194_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/C9314F1342D740C5B8C89FB12C55DBB8_20_16.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report195_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/CBA6D54FF6F347B4B0676331FD87295C_26_14.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report196_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/CBA6D54FF6F347B4B0676331FD87295C_27_13.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report197_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/CBA6D54FF6F347B4B0676331FD87295C_3_10.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report198_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/D2B4723C805C42748F405FDDD94E114C_24_15.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report199_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/D2B4723C805C42748F405FDDD94E114C_28_16.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report200_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/D2B4723C805C42748F405FDDD94E114C_6_8.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report201_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/D496C7CA2D9F48488AC516ADE41E839D_24_12.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report202_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/D496C7CA2D9F48488AC516ADE41E839D_28_7.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report203_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/D496C7CA2D9F48488AC516ADE41E839D_7_7.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report204_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/D4EE7DB0625E4DE6B7AE26B29F297607_24_11.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report205_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/D4EE7DB0625E4DE6B7AE26B29F297607_27_10.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report206_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/D4EE7DB0625E4DE6B7AE26B29F297607_7_10.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report207_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/D527009BB12344CD9DBE54B60AE06D98_11_8.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report208_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/D527009BB12344CD9DBE54B60AE06D98_17_11.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report209_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/D527009BB12344CD9DBE54B60AE06D98_28_8.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report210_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/D75473554303430AADBC77748F20E4E9_13_7.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report211_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/D75473554303430AADBC77748F20E4E9_19_12.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report212_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/D75473554303430AADBC77748F20E4E9_27_10.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report213_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/D8991C4D1D924B97998B81B4804A3A72_18_8.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report214_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/D8991C4D1D924B97998B81B4804A3A72_24_7.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report215_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/D8991C4D1D924B97998B81B4804A3A72_3_11.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report216_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/D8DDAC3FCAF04F2FAB556415E6548815_18_16.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report217_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/D8DDAC3FCAF04F2FAB556415E6548815_6_8.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report218_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/D8DDAC3FCAF04F2FAB556415E6548815_7_13.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report219_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/DBBD9A73B355472188EC58D66BDFF9C0_19_11.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report220_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/DBBD9A73B355472188EC58D66BDFF9C0_27_9.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report221_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/DBBD9A73B355472188EC58D66BDFF9C0_28_13.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report222_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/DE9680C86ACE45F083586D625595D07F_12_9.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report223_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/DE9680C86ACE45F083586D625595D07F_13_10.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report224_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/DE9680C86ACE45F083586D625595D07F_27_9.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report225_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/E5A01DE79553432FA7F86CAF28A8599B_13_11.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report226_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/E5A01DE79553432FA7F86CAF28A8599B_18_8.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report227_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/E5A01DE79553432FA7F86CAF28A8599B_20_7.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report228_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/E6B970EC95E444759FEF0581FC0110B1_17_15.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report229_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/E6B970EC95E444759FEF0581FC0110B1_25_16.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report230_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/E6B970EC95E444759FEF0581FC0110B1_4_8.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report231_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/E7FD916EDB4E4533B3A57E8A5053DAA8_18_10.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report232_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/E7FD916EDB4E4533B3A57E8A5053DAA8_20_9.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report233_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/E7FD916EDB4E4533B3A57E8A5053DAA8_24_16.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report234_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/ED5760AB46DC44BBAB0CF9E8F59658D5_11_16.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report235_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/ED5760AB46DC44BBAB0CF9E8F59658D5_4_12.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report236_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/ED5760AB46DC44BBAB0CF9E8F59658D5_5_7.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report237_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/EEFC922BA6B543A28FECDEC638E07EE0_20_10.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report238_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/EEFC922BA6B543A28FECDEC638E07EE0_20_13.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report239_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/EEFC922BA6B543A28FECDEC638E07EE0_26_8.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report240_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/F1EE0B9AD5F7421EB3B49578A2E3B09B_17_7.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report241_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/F1EE0B9AD5F7421EB3B49578A2E3B09B_28_9.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report242_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/F1EE0B9AD5F7421EB3B49578A2E3B09B_7_13.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report243_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/F1F03A18392E40818B01E55D19C26700_20_13.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report244_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/F1F03A18392E40818B01E55D19C26700_24_7.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report245_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/F1F03A18392E40818B01E55D19C26700_6_16.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report246_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/F7163B9D8FB64D1F828028906A69CD98_13_15.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report247_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/F7163B9D8FB64D1F828028906A69CD98_14_13.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report248_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/F7163B9D8FB64D1F828028906A69CD98_20_16.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report249_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/F94C741BE5064831A356F30C6863C6E2_13_16.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report250_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/F94C741BE5064831A356F30C6863C6E2_6_12.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report251_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/F94C741BE5064831A356F30C6863C6E2_7_12.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report252_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/F9A6820A57E542CBA302680BB6BD2B22_24_10.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report253_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/F9A6820A57E542CBA302680BB6BD2B22_6_13.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report254_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/F9A6820A57E542CBA302680BB6BD2B22_7_14.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report255_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/FAF3539D627A4E6280B60BC3C73E95E2_11_13.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report256_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/FAF3539D627A4E6280B60BC3C73E95E2_13_9.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report257_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/FAF3539D627A4E6280B60BC3C73E95E2_6_7.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report258_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/FCAE13D8ADD94DB8B1BD65E4607B7DE0_13_9.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report259_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/FCAE13D8ADD94DB8B1BD65E4607B7DE0_18_10.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report260_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
        data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/hed_test_data/Meeting_Attendance_Report/FCAE13D8ADD94DB8B1BD65E4607B7DE0_4_14.json').text
        oea.land(data, 'graph_api/v1.0/meeting_attendance_report', 'report261_hed_test_data.json', oea.ADDITIVE_BATCH_DATA, currentDateTime)
    else:
        logger.info('No such dataset: enter either k12 or hed for the testdataSet parameter.')

## 2.) Pre-process Meeting Attendance Report

This function is used to avoid ingestion errors of the meeting_attendance_report table.

```method```: **preprocess_meeting**

 - If ```testdataSet``` is ```k12```, this function is not necessary since there isn't test data for the meeting_attendance_report table in K-12 set.
 - If ```testdataSet``` is ```hed```:
    * removes the '@odata.context' column from the meeting_attendance_report table pre-ingestion. Once the column is removed, the rest of the table overwrites the original meeting_attendance_report, and proceeds to ingest as normally.

In [ ]:
# b.) second function used. Pre-processes/cleans meeting_attendance_report before ingestion.
def preprocess_meeting_atten_report():
    # b.1) read in the original meeting_attendance_report table, remove the '@odata.context' column and confirm it has been removed.
    df = spark.read.format('json').load(oea.to_url('stage1/Transactional/graph_api/v1.0/meeting_attendance_report'), multiline='true')
    df_corrected = df.select('id', 'totalParticipantCount', 'meetingStartDateTime', 'meetingEndDateTime', 'attendanceRecords')
    # b.2) set the current date and time (using the correct format), and write out to the same relative same location, with a new rundate partition-folder.
    import datetime
    currentDate = datetime.datetime.now()
    currentDateTime = currentDate.strftime("%Y-%m-%d %H-%M-%S")
    table_path = 'stage1/Transactional/graph_api/v1.0/meeting_attendance_report/additive_batch_data/rundate=' + currentDateTime
    df_corrected.write.save(oea.to_url(table_path), format='json', mode='overwrite', overwriteSchema='true')
    # b.3) only house the latest rundate folder compared to the old data (which had the '@odata.context' column).
    def clean_data_lake_latest(source_path):
        latest_folder = oea.get_latest_folder(source_path)
        items = mssparkutils.fs.ls(oea.to_url(source_path))
        for item in items:
            if item.name != latest_folder:
                logger.info('file removal path: ' + item.path + ' with item: ' + item.name)
                oea.rm_if_exists(source_path + '/' + item.name)
                logger.info('Successfully removed folder: ' + item.name + ' from path: ' + item.path)
            else:
                logger.info('Kept folder: ' + item.name + ' from path: ' + item.path)
        logger.info('Finished cleaning data lake to house only the latest folder')
    clean_data_lake_latest('stage1/Transactional/graph_api/v1.0/meeting_attendance_report/additive_batch_data')
    # b.4) ad hoc work - remove the _SUCCESS file, otherwise this will throw an error when ingesting the table.
    table_path = 'stage1/Transactional/graph_api/v1.0/meeting_attendance_report/additive_batch_data/rundate=' + currentDateTime
    oea.rm_if_exists(table_path + '/_SUCCESS', False)

## 3.) Correct Schemas for Graph Dataset

This function is used to flatten the originally-nested arrays in the JSON files, corrects column dtypes as needed, and adds a column for unique primary keys as needed in the m365, teams, and meeting tables.

```method```: **correct_schemas**

 - If ```testdataSet``` is ```k12```:
   * **_correct_users_table**: flattens the users table.
   * **_correct_m365_table**: flattens, corrects some column dtypes, and adds a primary key per row for the m365_app_user_detail table.
   * **_correct_teams_table**: flattens, corrects some column dtypes, and adds a primary key per row some column dtypes for the teams_activity_user_detail table.
 - If ```testdataSet``` is ```hed```:
   * Same 3 functions as above, plus
   * **_correct_meetings_table**: flattens, corrects some column dtypes, and adds a primary key per row for the meeting_attendance_report table.

In [ ]:
# c.) third function used. Schema correction, since Graph data initially landed is unstructured with nested arrays, incorrect column dtypes, and without primary keys for 3 of the 4 tables.
def _correct_users_table(df):
    df_flat = df.select(F.explode('value').alias('exploded_values')).select("exploded_values.*")
    return df_flat

def _correct_m365_table(df):
    df_flat = df.select(F.explode('value').alias('exploded_values'), 'rundate').select("exploded_values.*", 'rundate')
    df_flat = df_flat.withColumn('reportPeriod', F.explode(F.col('details').reportPeriod)) \
                    .withColumn('mobile', F.explode(F.col('details').mobile)) \
                    .withColumn('web', F.explode(F.col('details').web)) \
                    .withColumn('mac', F.explode(F.col('details').mac)) \
                    .withColumn('windows', F.explode(F.col('details').windows)) \
                    .withColumn('excel', F.explode(F.col('details').excel)) \
                    .withColumn('excelMobile', F.explode(F.col('details').excelMobile)) \
                    .withColumn('excelWeb', F.explode(F.col('details').excelWeb)) \
                    .withColumn('excelMac', F.explode(F.col('details').excelMac)) \
                    .withColumn('excelWindows', F.explode(F.col('details').excelWindows)) \
                    .withColumn('oneNote', F.explode(F.col('details').oneNote)) \
                    .withColumn('oneNoteMobile', F.explode(F.col('details').oneNoteMobile)) \
                    .withColumn('oneNoteWeb', F.explode(F.col('details').oneNoteWeb)) \
                    .withColumn('oneNoteMac', F.explode(F.col('details').oneNoteMac)) \
                    .withColumn('oneNoteWindows', F.explode(F.col('details').oneNoteWindows)) \
                    .withColumn('outlook', F.explode(F.col('details').outlook)) \
                    .withColumn('outlookMobile', F.explode(F.col('details').outlookMobile)) \
                    .withColumn('outlookWeb', F.explode(F.col('details').outlookWeb)) \
                    .withColumn('outlookMac', F.explode(F.col('details').outlookMac)) \
                    .withColumn('outlookWindows', F.explode(F.col('details').outlookWindows)) \
                    .withColumn('powerPoint', F.explode(F.col('details').powerPoint)) \
                    .withColumn('powerPointMobile', F.explode(F.col('details').powerPointMobile)) \
                    .withColumn('powerPointWeb', F.explode(F.col('details').powerPointWeb)) \
                    .withColumn('powerPointMac', F.explode(F.col('details').powerPointMac)) \
                    .withColumn('powerPointWindows', F.explode(F.col('details').powerPointWindows)) \
                    .withColumn('teams', F.explode(F.col('details').teams)) \
                    .withColumn('teamsMobile', F.explode(F.col('details').teamsMobile)) \
                    .withColumn('teamsWeb', F.explode(F.col('details').teamsWeb)) \
                    .withColumn('teamsMac', F.explode(F.col('details').teamsMac)) \
                    .withColumn('teamsWindows', F.explode(F.col('details').teamsWindows)) \
                    .withColumn('word', F.explode(F.col('details').word)) \
                    .withColumn('wordMobile', F.explode(F.col('details').wordMobile)) \
                    .withColumn('wordWeb', F.explode(F.col('details').wordWeb)) \
                    .withColumn('wordMac', F.explode(F.col('details').wordMac)) \
                    .withColumn('wordWindows', F.explode(F.col('details').wordWindows)) \
                    .drop('details')
    # temp: add unique primary key per row, by combining UPNs with the date the report was generated
    # this assumes every person has only one row per reportRefreshDate
    df_flat = df_flat.withColumn('m365Activity_pk', F.concat(F.col('userPrincipalName'),F.lit('_'),F.col('reportRefreshDate')))

    df_flat.select(F.col('reportRefreshDate'), F.to_date(F.col('reportRefreshDate'), 'yyyy-MM-dd'))
    df_flat.select(F.col('lastActivityDate'), F.to_date(F.col('lastActivityDate'), 'yyyy-MM-dd'))
    df_flat.select(F.col('lastActivationDate'), F.to_date(F.col('lastActivationDate'), 'yyyy-MM-dd'))
    return df_flat

def _correct_teams_table(df):
    df_flat = df.select(F.explode('value').alias('exploded_values'), 'rundate').select("exploded_values.*", 'rundate')
    df_flat = df_flat.withColumn('assignedProducts', F.explode(F.col('assignedProducts')))
    # convert duration to seconds only 
    # NOTE: The duration expression may have changed and this will need to be modified to accommodate any new duration formatting
    df_flat = df_flat.withColumn(
        'screenShareDuration', 
        F.coalesce(F.regexp_extract('screenShareDuration', r'(\d+)H', 1).cast('int'), F.lit(0)) * 3600 + 
        F.coalesce(F.regexp_extract('screenShareDuration', r'(\d+)M', 1).cast('int'), F.lit(0)) * 60 + 
        F.coalesce(F.regexp_extract('screenShareDuration', r'(\d+)S', 1).cast('int'), F.lit(0))
        ).withColumn(
        'videoDuration', 
        F.coalesce(F.regexp_extract('videoDuration', r'(\d+)H', 1).cast('int'), F.lit(0)) * 3600 + 
        F.coalesce(F.regexp_extract('videoDuration', r'(\d+)M', 1).cast('int'), F.lit(0)) * 60 + 
        F.coalesce(F.regexp_extract('videoDuration', r'(\d+)S', 1).cast('int'), F.lit(0))
        ).withColumn(
        'audioDuration', 
        F.coalesce(F.regexp_extract('audioDuration', r'(\d+)H', 1).cast('int'), F.lit(0)) * 3600 + 
        F.coalesce(F.regexp_extract('audioDuration', r'(\d+)M', 1).cast('int'), F.lit(0)) * 60 + 
        F.coalesce(F.regexp_extract('audioDuration', r'(\d+)S', 1).cast('int'), F.lit(0))
        )
    # temp: add unique primary key per row, by combining UPNs with the date the report was generated
    # this assumes every person has only one row per reportRefreshDate
    df_flat = df_flat.withColumn('teamsActivity_pk', F.concat(F.col('userPrincipalName'),F.lit('_'),F.col('reportRefreshDate')))

    df_flat.select(F.col('reportRefreshDate'), F.to_date(F.col('reportRefreshDate'), 'yyyy-MM-dd'))
    df_flat.select(F.col('lastActivityDate'), F.to_date(F.col('lastActivityDate'), 'yyyy-MM-dd'))
    return df_flat

def _correct_meetings_table(df):
    df_flat = df.select(
        "id", "meetingEndDateTime", "meetingStartDateTime", "totalParticipantCount",
        F.explode("attendanceRecords").alias("attendanceRecordsExplode"), 'rundate'
        ).select("id", "meetingEndDateTime", "meetingStartDateTime", "totalParticipantCount", 
                "attendanceRecordsExplode.*", 'rundate')
    df_flat = df_flat.withColumnRenamed("id","meetingId")
    df_flat = df_flat.select(
        "meetingId", "meetingEndDateTime", "meetingStartDateTime", "totalParticipantCount", 
        "totalAttendanceInSeconds", "role", "emailAddress", "attendanceIntervals",
        F.explode(F.array("identity")).alias("identityExplode"), 'rundate'
        ).select("meetingId", "meetingEndDateTime", "meetingStartDateTime", "totalParticipantCount", 
            "totalAttendanceInSeconds", "role", "emailAddress","attendanceIntervals",
            "identityExplode.*", 'rundate')

    df_flat = df_flat.select(
        "meetingId", "meetingEndDateTime", "meetingStartDateTime", "totalParticipantCount", 
        "totalAttendanceInSeconds", "role", "emailAddress",
        "displayName", "id", "tenantId",
        F.explode("attendanceIntervals").alias("attendanceIntervalsExplode"), 'rundate'
        ).select("meetingId", "meetingEndDateTime", "meetingStartDateTime", "totalParticipantCount", 
            "totalAttendanceInSeconds", "role", "emailAddress",
            "attendanceIntervalsExplode.*",
            "displayName", "id", "tenantId", 'rundate')
    # clean up column names and timestamp types
    df_flat = df_flat.withColumnRenamed("id", "userId").withColumnRenamed("displayName", "userDisplayName").withColumnRenamed("emailAddress", "userEmailAddress") \
        .withColumnRenamed("totalAttendanceInSeconds", "totalAttendanceInSec").withColumnRenamed("tenantId", "userTenantId") \
        .withColumnRenamed("joinDateTime", "attendanceInterval_joinDateTime").withColumnRenamed("leaveDateTime", "attendanceInterval_leaveDateTime").withColumnRenamed("durationInSeconds", "attendanceInterval_durationInSec")
    df_flat = df_flat.withColumn('meetingStartDateTime', F.to_timestamp(F.col('meetingStartDateTime'))) \
                .withColumn('meetingEndDateTime', F.to_timestamp(F.col('meetingEndDateTime'))) \
                .withColumn('attendanceInterval_joinDateTime', F.to_timestamp(F.col('attendanceInterval_joinDateTime'))) \
                .withColumn('attendanceInterval_leaveDateTime', F.to_timestamp(F.col('attendanceInterval_leaveDateTime')))
    # temp: add unique primary key per row, by combining meeting IDs with user IDs
    # this assumes every person attending a meeting has only one attendance interval
    df_flat = df_flat.withColumn('meetingUserId_pk', F.concat(F.col('meetingId'),F.col('userId')))
    df_flat = df_flat.select(
        'meetingUserId_pk','meetingId','meetingStartDateTime','meetingEndDateTime','totalParticipantCount','userId','userDisplayName','userEmailAddress',
        'userTenantId','role','totalAttendanceInSec','attendanceInterval_joinDateTime','attendanceInterval_leaveDateTime','attendanceInterval_durationInSec','rundate'
        )
    return df_flat

def correct_graph_dataset(tables_source):
    items = oea.get_folders(tables_source)
    for item in items: 
        table_path = tables_source +'/'+ item
        if item == 'metadata.csv':
            logger.info('ignore metadata processing, since this is not a table to be ingested')
        elif item == 'users':
            df_users = spark.read.format('delta').load(oea.to_url(table_path), header='true')
            df_corrected = _correct_users_table(df_users)
            df_corrected.write.save(oea.to_url(table_path), format='delta', mode='overwrite', header='true', overwriteSchema='true')
            logger.info('Successfully corrected the users table from: ' + table_path)
        elif item == 'm365_app_user_detail':
            df_m365 = spark.read.format('delta').load(oea.to_url(table_path), header='true')
            df_corrected = _correct_m365_table(df_m365)
            df_corrected.write.save(oea.to_url(table_path), format='delta', mode='overwrite', header='true', overwriteSchema='true')
            logger.info('Successfully corrected the m365_app_user_detail table from: ' + table_path)
        elif item == 'teams_activity_user_detail':
            df_teams = spark.read.format('delta').load(oea.to_url(table_path), header='true')
            df_corrected = _correct_teams_table(df_teams)
            df_corrected.write.save(oea.to_url(table_path), format='delta', mode='overwrite', header='true', overwriteSchema='true')
            logger.info('Successfully corrected the teams_activity_user_detail table from: ' + table_path)
        elif item == 'meeting_attendance_report':
            df_meetings = spark.read.format('delta').load(oea.to_url(table_path), header='true')
            df_corrected = _correct_meetings_table(df_meetings)
            df_corrected.write.save(oea.to_url(table_path), format='delta', mode='overwrite', header='true', overwriteSchema='true')
            logger.info('Successfully corrected the meeting_attendance_report table from: ' + table_path)
        else:
            logger.info('No defined function for table: ' + item)
    logger.info('Finished schema correction for Graph dataset')

## 4.) Refine Graph Dataset

This function is used to speed up the refining of the dataset.

```method```: **refine_graph**

 - If ```testdataSet``` is ```k12```:
   * Refine the 3 tables from the Graph beta query.
 - If ```testdataSet``` is ```hed```:
   * Refine the 3 tables from the Graph beta query and refine the meeting_attendance_report table from the Graph v1.0 query.

In [ ]:
# d.) fourth fuction used. This step refines the data through the use of metadata (this is where the pseudonymization of the data occurs).
def refine_graph_dataset(tables_source):
    items = oea.get_folders(tables_source)
    for item in items: 
        table_path = tables_source +'/'+ item
        if item == 'metadata_beta.csv':
            logger.info('ignore metadata processing, since this is not a table to be ingested')
        elif item == 'metadata_v1p0.csv':
            logger.info('ignore metadata processing, since this is not a table to be ingested')
        else:
            try:
                if item == 'users':
                    oea.refine('graph_api/beta/' + item, metadata_beta[item], 'userPrincipalName_pseudonym')
                elif item == 'm365_app_user_detail':
                    oea.refine('graph_api/beta/' + item, metadata_beta[item], 'm365Activity_pk_pseudonym')
                elif item == 'teams_activity_user_detail':
                    oea.refine('graph_api/beta/' + item, metadata_beta[item], 'teamsActivity_pk_pseudonym')
                elif item == 'meeting_attendance_report':
                    oea.refine('graph_api/v1.0/' + item, metadata_v1p0[item], 'meetingUserId_pk_pseudonym')
                else:
                    logger.info('No defined function for processing this Graph table.')
            except AnalysisException as e:
                # This means the table may have not been properly refined due to errors with the primary key not aligning with columns expected in the lookup table.
                pass
            
            logger.info('Refined table: ' + item + ' from: ' + table_path)

In [ ]:
if testdataSet == 'hed':
    if method == 'land_testdata':
        land_graph_testdata(testdataSet)
    elif method == 'preprocess_meeting':
        preprocess_meeting_atten_report()
    elif method == 'correct_schemas':
        correct_graph_dataset('stage2/Ingested/graph_api/beta')
        correct_graph_dataset('stage2/Ingested/graph_api/v1.0')
    elif method == 'refine_graph':
        #metadata_beta = oea.get_metadata_from_url('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/metadata_beta.csv')
        #metadata_v1p0 = oea.get_metadata_from_url('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/metadata_v1p0.csv')
        metadata_beta = oea.get_metadata_from_url('https://raw.githubusercontent.com/cstohlmann/oea-graph-api/main/test_data/metadata_beta.csv')
        metadata_v1p0 = oea.get_metadata_from_url('https://raw.githubusercontent.com/cstohlmann/oea-graph-api/main/test_data/metadata_v1p0.csv')
        refine_graph_dataset('stage2/Ingested/graph_api/beta')
        refine_graph_dataset('stage2/Ingested/graph_api/v1.0')
    else:
        logger.info('Method not defined - accepted methods for higher ed: land_testdata, preprocess_meeting, correct_schema, refine_graph')
    
elif testdataSet == 'k12':
    if method == 'land_testdata':
        land_graph_testdata(testdataSet)
    elif method == 'preprocess_meeting':
        logger.info('No meeting data to process: try using HEd for meeting_attendance_report test data.')
    elif method == 'correct_schemas':
        correct_graph_dataset('stage2/Ingested/graph_api/beta')
    elif method == 'refine_graph':
        #metadata_beta = oea.get_metadata_from_url('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Graph/test_data/metadata_beta.csv')
        metadata_beta = oea.get_metadata_from_url('https://raw.githubusercontent.com/cstohlmann/oea-graph-api/main/test_data/metadata_beta.csv')
        refine_graph_dataset('stage2/Ingested/graph_api/beta')
    else:
        logger.info('Method not defined - accepted methods for k-12: land_testdata, correct_schema, refine_graph')

else:
    logger.info('Unrecognized test dataset: input either k12 or hed for this parameter.')